# Deep Convolutional Generative Adversarial Networks (DCGANs) on the Fashion MNIST dataset using Keras and TensorFlow

In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape

/home/mfayemiwo/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mfayemiwo/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mfayemiwo/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mfayemiwo/anaconda3/lib/python3.6/site-packages/tensorflow/python/f

In [2]:
class DCGAN:
    @staticmethod
    def build_generator(dim, depth, channels=1, inputDim=100,
        outputDim=512):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (dim, dim, depth)
        chanDim = -1
        
        # first set of FC => RELU => BN layers
        model.add(Dense(input_dim=inputDim, units=outputDim))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        # second set of FC => RELU => BN layers, this time preparing the number of FC nodes to be
        # reshaped into a volume
        model.add(Dense(dim * dim * depth))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        
        # reshape the output of the previous layer set, upsample + apply a transposed convolution, RELU, and BN
        model.add(Reshape(inputShape))
        model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2),
            padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        
        # apply another upsample and transposed convolution, but this time output the TANH activation
        model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2),
            padding="same"))
        model.add(Activation("tanh"))
        # return the generator model
        return model
    
    
    
    @staticmethod
    def build_discriminator(width, height, depth, alpha=0.2):
        # initialize the model along with the input shape to be "channels last"
        model = Sequential()
        inputShape = (height, width, depth)
        # first set of CONV => RELU layers
        model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2),
            input_shape=inputShape))
        model.add(LeakyReLU(alpha=alpha))
        # second set of CONV => RELU layers
        model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
        model.add(LeakyReLU(alpha=alpha))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=alpha))
        # sigmoid layer outputting a single value
        model.add(Dense(1))
        model.add(Activation("sigmoid"))
        # return the discriminator model
        return model

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from sklearn.utils import shuffle
from imutils import build_montages
from keras.preprocessing.image import img_to_array
import numpy as np
import argparse
import cv2
import os
from imutils import paths

Using TensorFlow backend.


In [4]:
# load the Fashion MNIST dataset and stack the training and testing data points so we have
# additional training data
print("[INFO] loading MNIST dataset...")
((trainX, _), (testX, _)) = fashion_mnist.load_data()
trainImages = np.concatenate([trainX, testX])
# add in an extra dimension for the channel and scale the images into the range [-1, 1]
# (which is the range of the tanh function)
trainImages = np.expand_dims(trainImages, axis=-1)
trainImages = (trainImages.astype("float") - 127.5) / 127.5

[INFO] loading MNIST dataset...


In [5]:
# store the epochs and batch size in convenience variables, then initialize our learning rate
NUM_EPOCHS = 5
BATCH_SIZE = 128
INIT_LR = 2e-4

In [6]:
# build the generator
print("[INFO] building generator...")
gen = DCGAN.build_generator(7, 64, channels=1)
# build the discriminator
print("[INFO] building discriminator...")
disc = DCGAN.build_discriminator(28, 28, 1)
discOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
disc.compile(loss="binary_crossentropy", optimizer=discOpt)

[INFO] building generator...
Instructions for updating:
Colocations handled automatically by placer.
[INFO] building discriminator...


In [7]:
# build the adversarial model by first setting the discriminator to
# *not* be trainable, then combine the generator and discriminator
# together
print("[INFO] building GAN...")
disc.trainable = False
ganInput = Input(shape=(100,))
ganOutput = disc(gen(ganInput))
gan = Model(ganInput, ganOutput)
# compile the GAN
ganOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
gan.compile(loss="binary_crossentropy", optimizer=discOpt)

[INFO] building GAN...


In [8]:
# randomly generate some benchmark noise so we can consistently
# visualize how the generative modeling is learning
print("[INFO] starting training...")
benchmarkNoise = np.random.uniform(-1, 1, size=(256, 100))
# loop over the epochs
for epoch in range(0, NUM_EPOCHS):
    # show epoch information and compute the number of batches per
    # epoch
    print("[INFO] starting epoch {} of {}...".format(epoch + 1,
        NUM_EPOCHS))
    batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)
    # loop over the batches
    for i in range(0, batchesPerEpoch):
        # initialize an (empty) output path
        p = None
        # select the next batch of images, then randomly generate
        # noise for the generator to predict on
        imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
        
        # generate images using the noise + generator model
        genImages = gen.predict(noise, verbose=0)
        # concatenate the *actual* images and the *generated* images,
        # construct class labels for the discriminator, and shuffle
        # the data
        X = np.concatenate((imageBatch, genImages))
        y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
        y = np.reshape(y, (-1,))
        (X, y) = shuffle(X, y)
        # train the discriminator on the data
        discLoss = disc.train_on_batch(X, y)
        
        # let's now train our generator via the adversarial model by
        # (1) generating random noise and (2) training the generator
        # with the discriminator weights frozen
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        fakeLabels = [1] * BATCH_SIZE
        fakeLabels = np.reshape(fakeLabels, (-1,))
        ganLoss = gan.train_on_batch(noise, fakeLabels)
        
        
        # check to see if this is the end of an epoch, and if so,
        # initialize the output path
        if i == batchesPerEpoch - 1:
            p = ["output", "epoch_{}_output.png".format(
                str(epoch + 1).zfill(4))]
        # otherwise, check to see if we should visualize the current
        # batch for the epoch
        else:
            # create more visualizations early in the training
            # process
            if epoch < 10 and i % 25 == 0:
                p = ["output", "epoch_{}_step_{}.png".format(
                    str(epoch + 1).zfill(4), str(i).zfill(5))]
             # visualizations later in the training process are less
            # interesting
            elif epoch >= 10 and i % 100 == 0:
                p = ["output", "epoch_{}_step_{}.png".format(
                    str(epoch + 1).zfill(4), str(i).zfill(5))]
                
                
        # check to see if we should visualize the output of the
        # generator model on our benchmark data
        if p is not None:
            # show loss information
            print("[INFO] Step {}_{}: discriminator_loss={:.6f}, "
                "adversarial_loss={:.6f}".format(epoch + 1, i,
                    discLoss, ganLoss))
            # make predictions on the benchmark noise, scale it back
            # to the range [0, 255], and generate the montage
            images = gen.predict(benchmarkNoise)
            images = ((images * 127.5) + 127.5).astype("uint8")
            images = np.repeat(images, 3, axis=-1)
            vis = build_montages(images, (28, 28), (16, 16))[0]
            # write the visualization to disk
            p = os.path.sep.join(p)
            cv2.imwrite(p, vis)

[INFO] starting training...
[INFO] starting epoch 1 of 5...
Instructions for updating:
Use tf.cast instead.
[INFO] Step 1_0: discriminator_loss=0.697938, adversarial_loss=0.643400
[INFO] Step 1_25: discriminator_loss=0.046956, adversarial_loss=0.013209
[INFO] Step 1_50: discriminator_loss=0.000928, adversarial_loss=0.001270
[INFO] Step 1_75: discriminator_loss=0.000252, adversarial_loss=0.000780
[INFO] Step 1_100: discriminator_loss=0.000138, adversarial_loss=0.000669
[INFO] Step 1_125: discriminator_loss=0.000204, adversarial_loss=0.001067
[INFO] Step 1_150: discriminator_loss=0.000315, adversarial_loss=0.000597
[INFO] Step 1_175: discriminator_loss=0.000360, adversarial_loss=0.000288
[INFO] Step 1_200: discriminator_loss=0.000639, adversarial_loss=0.002250
[INFO] Step 1_225: discriminator_loss=0.098759, adversarial_loss=2.214174
[INFO] Step 1_250: discriminator_loss=0.205589, adversarial_loss=1.111233
[INFO] Step 1_275: discriminator_loss=0.322560, adversarial_loss=1.347596
[INFO] St

[INFO] Step 5_275: discriminator_loss=0.587682, adversarial_loss=1.076912
[INFO] Step 5_300: discriminator_loss=0.553686, adversarial_loss=0.873782
[INFO] Step 5_325: discriminator_loss=0.610920, adversarial_loss=0.756198
[INFO] Step 5_350: discriminator_loss=0.526347, adversarial_loss=0.728220
[INFO] Step 5_375: discriminator_loss=0.556878, adversarial_loss=1.044341
[INFO] Step 5_400: discriminator_loss=0.594185, adversarial_loss=0.865543
[INFO] Step 5_425: discriminator_loss=0.594914, adversarial_loss=0.833852
[INFO] Step 5_450: discriminator_loss=0.615390, adversarial_loss=1.130425
[INFO] Step 5_475: discriminator_loss=0.578268, adversarial_loss=1.076522
[INFO] Step 5_500: discriminator_loss=0.576386, adversarial_loss=0.963277
[INFO] Step 5_525: discriminator_loss=0.599806, adversarial_loss=0.952429
[INFO] Step 5_545: discriminator_loss=0.621665, adversarial_loss=1.025011
